# Introduction

This is the notebook responsible for calling the goodfire API.

We have a simple wrapper designed to form the right prompt and analyse the answer, and some utilities for running experiments over a range of parameters. All experimental results are dumped to csvs in the data/ folder, where they can be read by `analysis.ipynb`

# Setup

In [1]:
GOODFIRE_API_KEY = open("API_KEY.txt", "r").read().strip()

In [2]:
import goodfire
from goodfire import AsyncClient

client = AsyncClient(
    GOODFIRE_API_KEY
  )

# Instantiate a model variant
#base = "meta-llama/Meta-Llama-3-8B-Instruct"
#base = "meta-llama/Meta-Llama-3.1-70B-Instruct"
base = "meta-llama/Llama-3.3-70B-Instruct"
#base = "meta-llama/Meta-Llama-3.1-8B-Instruct"
variant = goodfire.Variant(base)


# Jailbreak the variant using feature "Requests to bypass ethical and moral constraints"
# This can be useful to reduce the frequency of refusals
#variant.set(client.features.list(ids=["3b0f849c-b42a-4ce3-a89e-326e6569d51a"]), -0.5)

feature_counts = {
    "meta-llama/Llama-3.3-70B-Instruct": 65536,
    "meta-llama/Meta-Llama-3.1-8B-Instruct": 65536,
    #65536  for 8B
}
feature_count = feature_counts[base]

# Constants

In [3]:
import json
import shared


#QuestionSource = "OUS"
QuestionSource = "GGB"
Questions = shared.get_questions(QuestionSource)

# How to use the questions in a chatbot prompt.
QuestionTemplate = "\"{0}\"\n\nPlease rate the preceeding statement on 1 to 7 point Likert scale with 1 - Strongly Disagree, 7 - Strongly Agree. Print a single digit then end the message."
MAX_WORKERS=3

In [ ]:
# Persona Nationality 
from shared import nationality_to_continent

nationalities = [k for k,_ in nationality_to_continent.items()]
genders = ['male', 'female']
ages = [f'{age} year old' for age in ['21', '30', '40', '50', '60', '75']]

PersonaQuestionTemplate = "Consider the following statement as a {1} person: "+QuestionTemplate
# OUS_PersonaTemplate = "Imagine you are a {1}.\n\n"+OUS_Template



# Utility Code

In [4]:
from dataclasses import dataclass
import re
from typing import Union
import concurrent.futures as futures
from pprint import pprint
import numpy as np
from scipy.special import softmax
import asyncio
import time


@dataclass
class Response:
    question: Union[int, str]
    score: float
    text: str
    logits: dict[int, float]
    mean: float
    stddev: float
    source: str

async def run_question(question: Union[int, str], model=None, persona=None, progress=None) -> Response:
    orig_question = question
    if isinstance(question, int):
        question = Questions[question]
    model = model or variant
    if persona:
        user_text = PersonaQuestionTemplate.format(question["statement"], persona)
    else:
        user_text = QuestionTemplate.format(question)
    messages = [
            # System prompt discourages refusal
            {"role": "system", "content": "Always answer the user's question."},
            {"role": "user", "content": user_text},
            # Encourages a single value response. Also discourages refusal?
            {"role": "assistant", "content": "I'd rate this statement: "}
        ]
    response = await client.chat.completions.create(
        messages,
        model=model,
        max_completion_tokens=50,
        temperature=0
    )
    text = response.choices[0].message["content"]
    score = None
    # Try some heuristics for finding the score
    match = (
        re.search(r"(\d) out of 7", text) or
        re.search(r"(\d)", text)
    )
    if match:
        try:
            score_text = match.group(1)
            score = int(score_text)
        except:
            pass

    logits = None
    mean = None
    stddev = None
    if score is not None:
        # Attempt to get logits
        logit_messages = messages + [{"role": "assistant", "content": match.string[:match.start(1)]}]
        logits = await client.chat.logits(logit_messages,
            model=model,
            top_k=100, #  has to be reasonably large so we don't drop anything significant
            filter_vocabulary=list('1234567')
        )
        logits = {int(k): v for k,v in logits.logits.items() if k in '1234567'}
        if logits:
            probs = dict(zip(logits.keys(), softmax(np.array(list(logits.values())))))
            mean = np.sum([k*v for k,v in probs.items()])
            stddev = np.sqrt(np.sum([v * (k - mean)**2 for k,v in probs.items()]))

    if progress:
        progress.update()
    return Response(question=orig_question, score=score, text=text, logits=logits, mean=mean, stddev=stddev, source=QuestionSource)

async def run_questions(*args, **kwargs) -> list[Response]:
        async with asyncio.TaskGroup() as tg:
            tasks = [tg.create_task(run_question(q, *args, **kwargs)) for q in range(len(Questions))]
        return [await task for task in tasks]
    
def to_vector(responses: list[Response]) -> np.array:
    return np.array([r.mean if r.mean is not None else np.nan for r in responses])

import datetime

def now_str():
    return datetime.datetime.now().strftime("%Y%m%d%H%M%S")

def clone(variant: goodfire.Variant) -> goodfire.Variant:
    new_variant = goodfire.Variant(variant.base_model)
    for edit in variant.edits:
        new_variant.set(edit[0], edit[1]['value'], mode=edit[1]['mode'])

    return new_variant

In [9]:
# Some testing
#q = run_question(1)
#print(q)
#qs = run_questions()
#pprint(qs)
#print(to_vector(qs))

In [5]:
from typing import Optional
import tqdm
import time
import pandas as pd

async def tabular_experiments(features: list[goodfire.Feature], steerages: list[float], personas: Optional[list[str]] = None, wait: Optional[float]=None, base=base):
    if personas is None:
        personas = [None]
    results = []
    async with asyncio.TaskGroup() as tg:
        combinations = []
        for feature in features:
            for steerage in steerages:
                model = goodfire.Variant(base)
                if feature is None:
                    assert steerage == 0
                else:
                    model.set(feature, steerage)
                for persona in personas:
                    combinations.append((feature, steerage, persona))
        tasks = []
        progress = tqdm.tqdm(total=len(combinations) * len(Questions))
        for combination in combinations :
            feature, steerage, persona = combination
            task = tg.create_task(run_questions(persona=persona, model=model, progress=progress))
            tasks.append((feature, steerage, persona, task))
            # TODO: Remove once we get parallellism working better
            await task
            
        for feature, steerage, persona, task in tasks:
            responses: list[Response] = task.result()
            if wait:
                time.sleep(wait)
            for response in responses:
                results.append(dict(
                    base=base,
                    source=response.source,
                    feature=feature.label if feature else "",
                    steerage=steerage,
                    persona=persona,
                    question=response.question,
                    mean_score=response.mean,
                    stddev_score=response.stddev,
                    score=response.score,
                    text=response.text,
                ))
    return pd.DataFrame(results)

## Features of interest

In [6]:
moral_keywords = ['moral', 'altruism', 'greater good', 'ethic', 'integrity', 'dignity']

# Experiments

In [7]:
import time

start_time = time.time()

for keyword in moral_keywords[:1]:
    print(f'Running search and steering for features associated with "{keyword}"\n')
    features = list((await client.features.search(keyword, model=base, top_k=10)))
    steerages = [-.5, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.5]
    experiments = await tabular_experiments(features, steerages)
    experiments.to_csv("data/" + now_str()+''.join(keyword)+".csv", index=False)
    end_time = time.time()
    print(f'Time take for {keyword} -> {end_time-start_time}')


Running search and steering for features associated with "moral"



  0%|          | 0/8100 [00:00<?, ?it/s]

Start time: 1735649704.747242
Counting create: 1
Start time: 1735649705.763426
Counting create: 2
Start time: 1735649706.782007
Counting create: 3
Start time: 1735649707.795696
Counting create: 4
Start time: 1735649708.820061
Counting create: 5
Start time: 1735649709.8348029
Counting create: 6
Start time: 1735649710.854866
Counting create: 7
Start time: 1735649711.87549
Counting create: 8
Start time: 1735649712.897356
Counting create: 9
Start time: 1735649713.923336
Counting create: 10
Start time: 1735649714.9340851
Counting create: 11
Start time: 1735649715.95287
Counting create: 12
Start time: 1735649716.9696069
Counting create: 13
Start time: 1735649717.989024
Counting create: 14
Start time: 1735649719.004817
Counting create: 15
Start time: 1735649720.0225651
Counting create: 16
Start time: 1735649721.0444002
Counting create: 17
Start time: 1735649722.062031
Counting create: 18
Start time: 1735649723.083165
Counting create: 19
Start time: 1735649724.098978
Counting create: 20
Start 

  1%|          | 53/8100 [01:37<45:06,  2.97it/s]   

Start time: 1735649802.761447
Counting create: 91
Start time: 1735649803.780353
Counting create: 92
Start time: 1735649804.7950232
Counting create: 93
Start time: 1735649805.8109848
Counting create: 94
Start time: 1735649806.8320708
Counting create: 95
Start time: 1735649807.8517241
Counting create: 96
Start time: 1735649808.867528
Counting create: 97
Start time: 1735649809.881768
Counting create: 98
Start time: 1735649810.89907
Counting create: 99
Start time: 1735649811.9242802
Counting create: 100
Start time: 1735649812.939115
Counting create: 101


...

  9%|▉         | 720/8100 [22:40<37:17,  3.30it/s]

Start time: 1735651064.5496252
Counting create: 738
Start time: 1735651065.5733492
Counting create: 739
Start time: 1735651066.5963678
Counting create: 740
Start time: 1735651067.61628
Counting create: 741
Start time: 1735651068.629686
Counting create: 742
Start time: 1735651069.644039
Counting create: 743
Start time: 1735651070.66013
Counting create: 744
Start time: 1735651071.685824
Counting create: 745
Start time: 1735651072.711371
Counting create: 746
Start time: 1735651073.735928
Counting create: 747
Start time: 1735651074.7608168
Counting create: 748
Start time: 1735651075.7756748
Counting create: 749
Start time: 1735651076.799975
Counting create: 750
Start time: 1735651077.8236558
Counting create: 751
Start time: 1735651078.845574
Counting create: 752
Start time: 1735651079.866115
Counting create: 753
Start time: 1735651080.877057
Counting create: 754
Start time: 1735651081.89746
Counting create: 755
Start time: 1735651082.916553
Counting create: 756
Start time: 1735651083.93438

Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...


End time: 1735651149.247426
Difference: 90.92439389228821
End time: 1735651149.255726
Difference: 30.59109115600586
End time: 1735651149.263388
Difference: 100.13553595542908
End time: 1735651149.2706292
Difference: 87.78786134719849
End time: 1735651149.2777731
Difference: 86.77980208396912
End time: 1735651149.28432
Difference: 40.84831714630127
End time: 1735651149.290571
Difference: 77.60474705696106
End time: 1735651149.296567
Difference: 72.49659204483032
End time: 1735651149.302482
Difference: 89.96217179298401
End time: 1735651149.3080819
Difference: 64.36121702194214
End time: 1735651149.313748
Difference: 99.15993285179138
End time: 1735651149.3195038
Difference: 82.72313594818115
End time: 1735651149.3254771
Difference: 98.15710115432739
End time: 1735651149.331552
Difference: 67.43409204483032
End time: 1735651149.3376
Difference: 52.15179705619812
End time: 1735651149.343335
Difference: 85.81481885910034
End time: 1735651149.3490949
Difference: 58.28965783119202
End time: 

Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
  9%|▉         | 721/8100 [24:19<5:53:36,  2.88s/it]Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit exceeded. Attempting exponential backoff...
Rate limit ex

In [ ]:
# Run baseline
if True:
    features = [None]
    steerages = [0]
    experiments = await tabular_experiments(features, steerages)
    experiments.to_csv("data/" + now_str()+".csv", index=False)

In [ ]:
# Run some random features
if False:
    features = list(client.features.search("elephants", model=base, top_k=1)[0])
    steerages = [-0.8, -0.5, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.5, 0.8]
    personas = [0]
    experiments = tabular_experiments(features, steerages, personas)
    experiments.to_csv("data/" + now_str()+".csv", index=False)

In [ ]:
# persona test
if False:
    features = list(client.features.search("moral", model=base, top_k=5)[0])
    steerages = [0]
    persona_tags = ['nationalities', 'ages', 'genders']
    for i, personas in enumerate([nationalities, ages, genders]):
        experiments = tabular_experiments(features[:1], steerages, personas)
        experiments.to_csv("data/" + now_str()+persona_tags[i]+".csv", index=False)

In [ ]:
import time
# keywords
#'overall impact','duty', 'dignity', 'greater good', git 
if False:
    for keyword in [#'obligation','ethic']: # 'dignity', 'greater good',
        'obligation','ethic']:
        print(f'Running search and steering for features associated with "{keyword}"\n')
        features = client.features.search(keyword, model=base)[0][:20]
        steerages = [-.5, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.5]
        experiments = tabular_experiments(features, steerages, personas=None, wait=1.5, base=base)
        experiments.to_csv("data/" + now_str()+''.join(keyword)+".csv", index=False)
        time.sleep(2)

In [ ]:
from itertools import batched
if False:
    for feature_ids in batched(range(0, feature_count), 20):
        features = client.features.lookup(list(feature_ids), model=base)
        print(features)


In [ ]:
# Experiment with logits
if False:
    logits = await client.chat.logits(
        messages=[
            {"role": "user", "content": "A random number between 0 and 9 is "}
        ],
        model="meta-llama/Llama-3.3-70B-Instruct",
        filter_vocabulary=list('0123456789')
    )
    print(logits.logits) 
    probs = dict(zip(logits.logits.keys(), softmax(np.array(list(logits.logits.values())))))
    print(probs)